# 국민재난안전포털 웹 크롤러 만들기

In [7]:
from requests import Session
from requests.compat import urljoin
from bs4 import BeautifulSoup

In [59]:
session = Session()

base_url = 'http://www.safekorea.go.kr/'
main_url = 'idsiSFK/neo/ext/json/main/menuList.json'

main_resp = session.get(urljoin(base_url, main_url))
result = main_resp.json()

disaster_service_url = ''
disaster_menu_seq = 1

# 재난문자 url과 params인 munuSeq 구하기
for cnt in range(0,len(result)):
    if cnt > 0 and result[cnt]['MENU_NM'] == '재난문자':
        disaster_service_url = 'idsiSFK' + result[cnt]['SERVICE_URL']
        disaster_menu_seq = result[cnt]['MENU_SEQ']
        break
        
#disaster_service_url, disaster_menu_seq

## 아래 코드 사용 불가
- post 방식의 재난문자 json request시 500 error 발생

In [75]:
session.cookies.clear()
disaster_resp = session.request('GET', urljoin(base_url, disaster_service_url), params={'menuSeq':disaster_menu_seq})
#dom = BeautifulSoup(disaster_resp.text, 'html.parser')
referer = disaster_resp.request.url

#새 쿠키값 
cookiedict = {}
for _ in session.cookies.items():
    cookiedict[_[0]] = _[1]

headers = {
    'Referer' : referer,
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}

#쿠키 날짜 value는 당일 날짜로 변경해야함, 실시간 크롤링시
cookies = {
    'JSESSIONID':cookiedict['JSESSIONID'],
    'elevisor_for_j2ee_uid':cookiedict['elevisor_for_j2ee_uid'],
    'e4d3f82b588e6cf132ee4765403cd800':cookiedict['e4d3f82b588e6cf132ee4765403cd800'],
    '3053195ebb01870828aed202f55dc974':'76ff93199833d41c33e1b89a9862350b',
    'popup_1':'20200802',
    'popup_2':'20200802',
    '_ga':'GA1.3.522710010.1595930482',
    '_gid':'GA1.3.1758721736.1596362546'
}

#session.request지만 누락된 쿠키값이 있어서 따로 다시 입력
disaster_list = 'idsiSFK/bbs/user/selectBbsList.do'
disaster_list_resp = session.request('POST', urljoin(base_url, disaster_list), headers=headers, cookies=cookies)

In [76]:
disaster_list_resp.request, disaster_list_resp.reason, disaster_list_resp.status_code, disaster_list_resp.headers

(<PreparedRequest [POST]>,
 'Internal Server Error',
 500,
 {'Content-Encoding': 'gzip', 'Content-Type': 'text/html;charset=ISO-8859-1', 'Content-Length': '254', 'Date': 'Sun, 02 Aug 2020 14:56:23 GMT'})

# 셀레니움 활용 실시간 문자 수집
- 셀레니움 사용 이유: 재난문자 페이지의 json request 불가(inter server error, 500 error) 

In [77]:
from selenium import webdriver

In [83]:
driver = webdriver.Chrome()
driver.implicitly_wait(3)

uu = urljoin(base_url, disaster_service_url)
uuu = uu + '&menuSeq=' + str(disaster_menu_seq)
driver.get(uuu)

In [99]:
# 첫 번째 페이지, 첫 번째 재난 문자 path
first_path = '//tbody[@id="bbs_tr"]/tr[1]/td[2]'
result_1 = driver.find_element_by_xpath(first_path)
print(result_1.text)

# 첫 번째 재난문자 링크 이동
first_a = first_path + '/a'
last_url = driver.current_url
driver.find_element_by_xpath(first_a).click()

2020/08/03 11:11:38 재난문자[춘천시재대본]


In [100]:
# 재난문자 상세내용 확인
disaster_context = '//div[@id="content"]/div[@id="content"]/div/div[@class="boardView"]/div[@class="boardView_context"]'
result_context = driver.find_element_by_xpath(disaster_context)
print(result_context.text)

# 이전 페이지로 돌아가기
driver.get(last_url)

[춘천시재대본] 현재 지역 곳곳에 많은 비가 내립니다. 산사태·침수 등 위험지역 대피, 외출자제 등 안전에 주의바랍니다.

-송출지역-
강원도 춘천시
